In [1]:

# %%
from attribution_utils import calculate_feature_attribution
from collections import defaultdict
from torch.nn.functional import log_softmax
from gemma_utils import get_all_string_min_l0_resid_gemma
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils
from functools import partial
import tqdm
from sae_lens import HookedSAETransformer, SAE, SAEConfig
from gemma_utils import get_gemma_2_config, gemma_2_sae_loader
import numpy as np
import torch
import tqdm
import pandas as pd

import torch.nn.functional as F
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:


model = HookedSAETransformer.from_pretrained("google/gemma-2-2b-it")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [3]:

generation_dict = torch.load("gemma2_generation_dict.pt")

## Filter non single token items

In [41]:

all_entropy_hyphen = []
all_entropy_breaks = []
tokens = []
for key in list(generation_dict.keys()):
    val = generation_dict[key]
    for toks in val:

In [46]:
print(break_pos)
print(hyphen_pos)
print(positions)

tensor([ 3, 26, 29, 32, 35, 38, 41, 45], device='cuda:0')
tensor([30, 33, 36, 39, 42], device='cuda:0')
[32, 35, 38, 41, 43]


In [4]:

def logits_to_entropy(logits):
    log_probs = log_softmax(logits, dim = -1)
    probs = log_probs.exp()
    entropy = -(log_probs*probs).sum(dim = -1)
    return entropy


def logits_to_varentropy(logits):
    log_probs = log_softmax(logits, dim = -1)
    probs = log_probs.exp()
    entropy = -(log_probs*probs).sum(dim = -1)
    elem = (probs*(-log_probs)**2).sum(dim = -1)
    return elem - entropy



In [ ]:

all_entropy_hyphen = []
tokens = []
for key in list(generation_dict.keys()):
    val = generation_dict[key]
    for toks in val:
        tokens.append(toks)
        with torch.no_grad():
            hyphen_pos = torch.where(toks[0] == 235290)[0]
            break_pos = torch.where(toks[0] == 108)[0]
            positions = (hyphen_pos[1:]-1).tolist() + [break_pos[-1].item()-2]
            logits = model(toks)
            entropy = logits_to_entropy(logits)
            all_entropy_hyphen.append(entropy[:,positions])



max_size = max(tensor.size(1) for tensor in all_entropy_hyphen)
padded_tensors = []
for tensor in all_entropy_hyphen:
    pad_amount = max_size - tensor.size(1)
    padded_tensor = F.pad(tensor, (pad_amount, 0), "constant", 0)  # Left padding
    padded_tensors.append(padded_tensor)
stacked_entropy_hyphen = torch.cat(padded_tensors,dim = 0)





In [48]:

px.imshow(stacked_entropy_hyphen.cpu().numpy(), aspect = 'auto')

In [ ]:

all_entropy_hyphen = []
tokens = []
for key in list(generation_dict.keys()):
    val = generation_dict[key]
    for toks in val:
        toks[0,8] = 1497
        tokens.append(toks)
        with torch.no_grad():
            hyphen_pos = torch.where(toks[0] == 235290)[0]
            break_pos = torch.where(toks[0] == 108)[0]
            positions = (hyphen_pos[1:]-1).tolist() + [break_pos[-1].item()-2]
            logits = model(toks)
            entropy = logits_to_entropy(logits)
            all_entropy_hyphen.append(entropy[:,positions])



max_size = max(tensor.size(1) for tensor in all_entropy_hyphen)
padded_tensors = []
for tensor in all_entropy_hyphen:
    pad_amount = max_size - tensor.size(1)
    padded_tensor = F.pad(tensor, (pad_amount, 0), "constant", 0)  # Left padding
    padded_tensors.append(padded_tensor)
stacked_entropy_hyphen = torch.cat(padded_tensors,dim = 0)


In [56]:

px.imshow(stacked_entropy_hyphen.cpu().numpy(), aspect = 'auto')

## Entropy of the long sequence

In [64]:

generation_dict = torch.load("gemma2_generation_long_dict.pt")
all_entropy_hyphen = []
tokens = []
for key in list(generation_dict.keys()):
    val = generation_dict[key]
    for toks in val:
        toks[0,8] = 1497
        tokens.append(toks)
        with torch.no_grad():
            hyphen_pos = torch.where(toks[0] == 235290)[0]
            break_pos = torch.where(toks[0] == 108)[0]
            positions = (hyphen_pos[1:]-1).tolist() + [break_pos[-1].item()-2]
            logits = model(toks)
            entropy = logits_to_entropy(logits)
            all_entropy_hyphen.append(entropy[:,positions])



max_size = max(tensor.size(1) for tensor in all_entropy_hyphen)
padded_tensors = []
for tensor in all_entropy_hyphen:
    pad_amount = max_size - tensor.size(1)
    padded_tensor = F.pad(tensor, (pad_amount, 0), "constant", 0)  # Left padding
    padded_tensors.append(padded_tensor)
stacked_entropy_hyphen = torch.cat(padded_tensors,dim = 0)

In [69]:
toks

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [65]:

px.imshow(stacked_entropy_hyphen.cpu().numpy(), aspect = 'auto')